In [ ]:
DATA_S3_PATH = 's3://mastering-ml-aws/chapter5/data.csv'
df = spark.read.csv(DATA_S3_PATH, header=True, inferSchema=True)


VBox()

In [8]:
df.show(5)

VBox()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

In [9]:
df = df.selectExpr("*","Quantity * UnitPrice as TotalBought")

VBox()

In [10]:
df.show(5)

VBox()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|       TotalBought|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
+---------+---------+---

In [11]:
customer_df = df.select("CustomerID","TotalBought").groupBy("CustomerID").sum("TotalBought").withColumnRenamed('sum(TotalBought)','SumTotalBought')

VBox()

In [12]:
from pyspark.sql.functions import *

joined_df = df.join(customer_df, 'CustomerId')

VBox()

In [13]:
joined_df.show(5)

VBox()

+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country|       TotalBought|   SumTotalBought|
+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+
|     17850|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|United Kingdom|15.299999999999999|5288.630000000009|
|     17850|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|United Kingdom|             20.34|5288.630000000009|
|     17850|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|United Kingdom|              22.0|5288.630000000009|
|     17850|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|United Kingdom|             20.34|5288.630000000009|
|     17850|   53636

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from sagemaker_pyspark import IAMRole
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator

role = 'arn:aws:iam::095585830284:role/EMR_EC2_DefaultRole'

kmeans_step = KMeansSageMakerEstimator(
    sagemakerRole=IAMRole(role),
    trainingInstanceType="ml.m4.xlarge",
    trainingInstanceCount=1,
    endpointInstanceType="ml.m4.xlarge",
    endpointInitialInstanceCount=1)
kmeans_step.setK(3)
kmeans_step.setFeatureDim(3722)

stages = [StringIndexer(inputCol='StockCode', outputCol="stock_code_index", handleInvalid='keep'),
          OneHotEncoder(inputCol='stock_code_index', outputCol='stock_code_encoded'),
          StringIndexer(inputCol='Country', outputCol='country_index', handleInvalid='keep'),
          OneHotEncoder(inputCol='country_index', outputCol='country_encoded'),
          QuantileDiscretizer(numBuckets=3, inputCol='SumTotalBought', outputCol='total_bought_index'),
          VectorAssembler(inputCols=['stock_code_encoded', 'country_encoded', 'total_bought_index'],
                          outputCol='features_raw'),
          Normalizer(inputCol="features_raw", outputCol="features", p=1.0),
          kmeans_step]

pipeline = Pipeline(stages=stages)

model = pipeline.fit(joined_df)


VBox()

Exception AttributeError: "'ScalaMap' object has no attribute '_java_obj'" in <bound method ScalaMap.__del__ of <sagemaker_pyspark.wrapper.ScalaMap object at 0x7f76b0280590>> ignored
Exception AttributeError: "'ScalaMap' object has no attribute '_java_obj'" in <bound method ScalaMap.__del__ of <sagemaker_pyspark.wrapper.ScalaMap object at 0x7f76b0280510>> ignored
Exception AttributeError: "'Option' object has no attribute '_java_obj'" in <bound method Option.__del__ of <sagemaker_pyspark.wrapper.Option object at 0x7f76b0155e50>> ignored
Exception AttributeError: "'Option' object has no attribute '_java_obj'" in <bound method Option.__del__ of <sagemaker_pyspark.wrapper.Option object at 0x7f76b0155850>> ignored
Exception AttributeError: "'Option' object has no attribute '_java_obj'" in <bound method Option.__del__ of <sagemaker_pyspark.wrapper.Option object at 0x7f76b0155e90>> ignored
Exception AttributeError: "'Option' object has no attribute '_java_obj'" in <bound method Option.__del_

In [21]:
df_with_clusters = model.transform(joined_df).cache()



VBox()

In [23]:
df_with_clusters.show(5)

VBox()

+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+----------------+------------------+-------------+---------------+------------------+--------------------+--------------------+-------------------+---------------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country|       TotalBought|   SumTotalBought|stock_code_index|stock_code_encoded|country_index|country_encoded|total_bought_index|        features_raw|            features|distance_to_cluster|closest_cluster|
+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+----------------+------------------+-------------+---------------+------------------+--------------------+--------------------+-------------------+---------------+
|     17850|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|Unit

In [24]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
evaluator.setPredictionCol("closest_cluster")

silhouette = evaluator.evaluate(df_with_clusters)


VBox()

In [25]:
silhouette

VBox()

0.34719788320926304